In [11]:
import json
import os

from datasets import Dataset, DatasetDict, load_dataset, load_from_disk
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

In [2]:
dataset_name = "NIKL_NEWSPAPER_2023_CSV"
fname = "NEWSPAPER_2022_1"
sample_name = "sample1"

source_dataset_dir = f"../dataset/{dataset_name}-{fname}-{sample_name}.parquet"

parquet_data_files = {
    "train": source_dataset_dir,
}


In [3]:
ds = load_dataset("parquet", data_files=parquet_data_files)

In [4]:
ds['train'][0]

{'file_id': 'NIRW2300000001',
 'doc_id': 'NIRW2300000001.52999',
 'title': '노컷뉴스 2022년 기사',
 'author': '최범규 충북CBS 최범규 기자',
 'publisher': '노컷뉴스',
 'date': 20221122,
 'topic': '사회',
 'original_topic': '지역>울산, 지역>대전, 지역>부산',
 'sentence_ids': '["NIRW2300000001.52999.1", "NIRW2300000001.52999.2", "NIRW2300000001.52999.3", "NIRW2300000001.52999.4", "NIRW2300000001.52999.5", "NIRW2300000001.52999.6", "NIRW2300000001.52999.7", "NIRW2300000001.52999.8", "NIRW2300000001.52999.9", "NIRW2300000001.52999.10", "NIRW2300000001.52999.11"]',
 'text': '맨몸으로 불난 집 뛰어들어 노인 구한 ‘슈퍼맨’, 정체 보니… 우체국 집배원이 맨몸으로 주택 화재 현장에 뛰어들어 70대 집주인을 구한 사실이 알려졌다. 주인공은 충북 영동군 영동우체국에서 근무하는 유지하(33)씨다. 유씨는 지난 21일 오전 11시쯤 영동읍 계산리 일대에서 우편물을 배달하던 중 한 주택에서 많은 연기가 치솟는 것을 목격했다. 급박한 상황을 직감한 유씨는 곧장 집으로 뛰어들어갔다. 집주인 70대 A씨가 평소 집에 머물고 있던 것을 알고 있던 유씨는 가장 먼저 A씨를 찾았다. 그러면서 집 뒤편에 있던 소화기로 진화하려다 쓰러져 있는 A씨를 발견했다. 유씨는 A씨를 업고 밖으로 나와 응급조치를 하며 119에 신고했다. 또 주변에 주차된 차량 탓에 소방차가 진입하기 어려울 것을 알고, 분주하게 움직이며 이동 주차도 요청했다. 유씨의 발빠른 대처에 A씨는 소중한 목숨을 구했고, 화재도 신속하게 진화됐다

In [5]:
# ds: 원본 전체 Dataset 라고 가정
# 1) 먼저 train : 나머지(=val+test)  = 0.8 : 0.2
tmp = ds['train'].train_test_split(test_size=0.2, seed=42)
train_ds = tmp["train"]          # 0.8
val_test = tmp["test"]           # 0.2

# 2) 나머지 0.2를 val:test = 0.5:0.5 로 나누면
#    최종 비율이 0.1 : 0.1 이 됨
val_test = val_test.train_test_split(test_size=0.5, seed=42)
val_ds = val_test["train"]       # 0.1
test_ds = val_test["test"]       # 0.

In [6]:
print(f"Train {len(train_ds)} Val {len(val_ds)} Test {len(test_ds)}")

Train 11992 Val 1499 Test 1499


In [7]:
test_ds

Dataset({
    features: ['file_id', 'doc_id', 'title', 'author', 'publisher', 'date', 'topic', 'original_topic', 'sentence_ids', 'text', 'entities'],
    num_rows: 1499
})

In [8]:
dataset_dict = DatasetDict({
    "train": train_ds,
    "validation": val_ds,
    "test": test_ds,
})

In [15]:
save_path = "./data/run1"

if not os.path.exists(save_path):
    os.makedirs(save_path)

In [16]:
train_ds.to_parquet(os.path.join(save_path, "train.parquet"))
val_ds.to_parquet(os.path.join(save_path, "val.parquet"))
test_ds.to_parquet(os.path.join(save_path, "test.parquet"))

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

5219401